<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo11s_960.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os
import shutil

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [4]:
# --- 縦横90°切替を学習に混ぜるコールバック ---
import torch, random

def _rotate_xywh_norm(xywh, k):
    x, y, w, h = xywh.unbind(-1)
    if k % 4 == 0:      xn, yn, wn, hn = x,       y,       w, h
    elif k % 4 == 1:    xn, yn, wn, hn = 1.0 - y, x,       h, w   # +90° CCW
    elif k % 4 == 2:    xn, yn, wn, hn = 1.0 - x, 1.0 - y, w, h   # 180°
    else:               xn, yn, wn, hn = y,       1.0 - x, h, w   # 270°
    return torch.stack([xn, yn, wn, hn], dim=-1)

def rotate_batch_90deg_callback(trainer):
    """学習バッチの開始時に、縦横90°/270°入替を確率pで適用"""
    p = 0
    if random.random() > p:
        return
    b = getattr(trainer, "batch", None)
    if b is None:
        return
    imgs = b.get("img", None)
    if imgs is None or imgs.ndim != 4:
        return
    k = random.choice([1, 3])  # 90° or 270°
    b["img"] = torch.rot90(imgs, k=k, dims=[-2, -1])
    if b.get("bboxes", None) is not None and b["bboxes"].numel() > 0:
        b["bboxes"] = _rotate_xywh_norm(b["bboxes"], k)

In [5]:
output_dir = "runs_experiment"  # runs_experimentが既に存在している場合削除
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print(f"✅ ディレクトリ '{output_dir}' を削除しました。")
else:
    print(f"⚠️ ディレクトリ '{output_dir}' は存在しません。")

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

⚠️ ディレクトリ 'runs_experiment' は存在しません。
Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [6]:
#!pip install -U torch torchvision torchaudio
!pip -q install ultralytics==8.3.177 --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00


In [7]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

sending incremental file list
dataset.zip
         13.37M 100%   27.01MB/s    0:00:00 (xfr#1, to-chk=0/1)
Archive:  /content/dataset.zip
   creating: /content/dataset/
  inflating: /content/dataset/data.yaml  
   creating: /content/dataset/images/
   creating: /content/dataset/images/test/
  inflating: /content/dataset/images/test/1f7de7eb-Ch1_20250910_095630.jpg  
  inflating: /content/dataset/images/test/691afc3e-Ch1_20250910_095611.jpg  
  inflating: /content/dataset/images/test/85dce6e3-Ch1_20250910_095624.jpg  
   creating: /content/dataset/images/train/
  inflating: /content/dataset/images/train/0e0f3b4f-Ch1_20250910_095659.jpg  
  inflating: /content/dataset/images/train/104868da-Ch1_20250910_095702.jpg  
  inflating: /content/dataset/images/train/1378a66f-Ch1_20250910_095634.jpg  
  inflating: /content/dataset/images/train/35e66149-Ch1_20250910_095646.jpg  
  inflating: /content/dataset/images/train/370db9b6-Ch1_20250910_095640.jpg  
  inflating: /content/dataset/images/train/3

In [8]:
# YAML
import yaml

# YAMLファイルを読み込む
data_yaml = "/content/dataset/data.yaml"
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [9]:
# yolo学習用関数（callbacks= を train() に渡さない）
from ultralytics import YOLO
import os

def train_yolo(
    model_path,
    data_yaml,
    epochs=200,
    batch_size=16,
    img_size=1280,
    output_dir="runs_experiment",
    iou_nms=0.9,
    mosaic=0.15, mixup=0.05, copy_paste=0.6, close_mosaic=15,
    degrees=5.0, shear=0.0, perspective=0.0, translate=0.05, scale=0.2,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    erasing=0.1, fliplr=0.5, flipud=0.0,
    lr0=0.004,
    optimizer="AdamW",
    cos_lr=True,
    warmup_epochs=3,
    amp=True,
    workers=0,
    project_name="yolo_project",
    **kwargs,
):
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # ← ここでコールバックを登録（train()には渡さない）回転コールバックは使用しない
    #model.add_callback("on_train_batch_start", rotate_batch_90deg_callback)

    os.makedirs(output_dir, exist_ok=True)
    print(f"Training YOLO with {epochs} epochs…")

    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,

        optimizer=optimizer, lr0=lr0, cos_lr=cos_lr, warmup_epochs=warmup_epochs,

        mosaic=mosaic, mixup=mixup, copy_paste=copy_paste, close_mosaic=close_mosaic,
        degrees=degrees, shear=shear, perspective=perspective,
        translate=translate, scale=scale,
        hsv_h=hsv_h, hsv_s=hsv_s, hsv_v=hsv_v,
        erasing=erasing, fliplr=fliplr, flipud=flipud,

        iou=iou_nms,  # ← iou_nms → iou

        amp=amp, workers=workers,
        project=output_dir, name=project_name,

        **kwargs,  # cache, rect, patience などはここで渡す
    )

    print(f"Training complete. Results saved to {output_dir}/{project_name}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [10]:
# ====== 実行パラメータ ======
model_path = "/content/drive/MyDrive/yolo11s.pt"  # or "/content/drive/MyDrive/last.pt"
data_yaml = "/content/dataset/data.yaml"
project_name = "yolo11s_960_learning"
output_dir = "runs_experiment"

train_params = {
    "model_path": model_path,
    "data_yaml": data_yaml,
    "epochs": 300,
    "batch_size": 32,
    "lr0": 0.004,
    "img_size": 960,
    "output_dir": output_dir,
    "cache": "ram",
    "rect": True,
    "iou_nms": 0.60,
    "mosaic": 0.0, "mixup": 0.0, "copy_paste": 0.0, "close_mosaic": 0,
    "degrees": 85.0, "shear": 0.0, "perspective": 0.0, "translate": 0.03, "scale": 0.2,
    "hsv_h": 0.015, "hsv_s": 0.4, "hsv_v": 0.2,
    "erasing": 0.1, "fliplr": 0.2, "flipud": 0.0,
    "optimizer": "AdamW",
    "cos_lr": True,
    "warmup_epochs": 3,
    "amp": True,
    "workers": 2,
    "project_name": project_name,
    # "callbacks": callbacks  ← ここは削除！
}

train_yolo(**train_params)


Loading model from /content/drive/MyDrive/yolo12s.pt
Training YOLO with 300 epochs…
New https://pypi.org/project/ultralytics/8.3.197 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.177 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset/data.yaml, degrees=85.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.1, exist_ok=False, fliplr=0.2, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.2, imgsz=960, int8=False, iou=0.6, keras=False, kobj=1.0, line_width=None, lr0=0.004, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDriv

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  2    689408  ultralytics.nn.modules.block.A2C2f           [256, 256, 2, True, 4]        
  7                  -1  1   1180672  ultralytics

  8                  -1  2   2689536  ultralytics.nn.modules.block.A2C2f           [512, 512, 2, True, 1]        
  9                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 10             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 11                  -1  1    345856  ultralytics.nn.modules.block.A2C2f           [768, 256, 1, False, -1]      
 12                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 13             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 14                  -1  1     95104  ultralytics.nn.modules.block.A2C2f           [512, 128, 1, False, -1]      
 15                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 16            [-1, 11]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3626.5±1154.6 MB/s, size: 581.3 KB)


train: Scanning /content/dataset/labels/train... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<00:00, 1450.00it/s]

...

    110/300      14.4G       1.04     0.6148      1.603        167        960: 100%|██████████| 1/1 [00:00<00:00,  4.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 17.80it/s]

                   all          4         40      0.181       0.37      0.186     0.0397
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 10, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



110 epochs completed in 0.027 hours.
Optimizer stripped from runs_experiment/yolo12s_960_learning/weights/last.pt, 19.0MB
Optimizer stripped from runs_experiment/yolo12s_960_learning/weights/best.pt, 19.0MB

Validating runs_experiment/yolo12s_960_learning/weights/best.pt...
Ultralytics 8.3.177 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 22.38it/s]


                   all          4         40      0.627          1      0.991      0.685
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs_experiment/yolo12s_960_learning
Training complete. Results saved to runs_experiment/yolo12s_960_learning


In [11]:
#output_dir = "runs_experiment"
best_model_path = f"{output_dir}/{project_name}/weights/best.pt"
last_model_path = f"{output_dir}/{project_name}/weights/last.pt"

from google.colab import files
import os

for model_path in [best_model_path, last_model_path]:
    if os.path.exists(model_path):
        print(f"Downloading: {model_path}")
        files.download(model_path)
    else:
        print(f"[ERROR] {os.path.basename(model_path)} が見つかりません: {model_path}")



Downloading: runs_experiment/yolo12s_960_learning/weights/best.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloading: runs_experiment/yolo12s_960_learning/weights/last.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>